In [100]:
%matplotlib inline
import re
import functools

In [349]:
d = pd.read_excel('/Users/eczech/data/ptbo2/pbto2_20151023_clean.xlsx', None)

In [350]:
is_num = lambda x: re.compile('^\d+$').match(x) is not None
d_sheet = [k for k in d.keys() if is_num(k)]
o_sheet = [k for k in d.keys() if k not in d_sheet]
len(ids), o_sheet

(339, ['BO2&Injy time', 'Demographics', 'Marsh & 12mGOS', 'Outcome'])

## Data Exploration

In [351]:
trans = {
    'right_pupil_response': 'pupil_response_r',
    'left_pupil_response' : 'pupil_response_l',
    'right_pupil_size'    : 'pupil_size_r',
    'left_pupil_size'     : 'pupil_size_l',
    'icp_1'               : 'icp1'
}
def normalize_col(c):
    c = c.strip().lower().replace(' ', '_').replace(',', '')
    return trans.get(c, c)

In [352]:
# Find the unique set of all columns that appear in data sheets
col_lists = [[normalize_col(c) for c in d[k].columns.tolist()] for k in d_sheet]
pd.Series(pd.DataFrame(col_lists).values.ravel()).order().unique()

array(['base_deficit_arterial', 'base_excess_arterial', 'blood_gases',
       'date', 'date/time', 'hco3', 'hco3a', 'icp', 'icp1', 'icp2', 'map',
       'paco2', 'pao2', 'pbto2', 'pha', 'pupil_response',
       'pupil_response_l', 'pupil_response_r', 'pupil_size',
       'pupil_size_l', 'pupil_size_r', 'sao2', 'spo2', 'uid',
       'unnamed:_14', 'unnamed:_19', 'vital_signs', None], dtype=object)

In [353]:
def find_sheets_by_col(col, first_only=False):
    """ Utility function for searching for sheets based on a column name"""
    res = []
    for k in d_sheet:
        df = d[k].copy()
        df.columns = [normalize_col(c) for c in df]
        df['sheet_name'] = int(k)
        if col is None or normalize_col(col) in df:
            if first_only:
                return df
            res.append(df)
    return functools.reduce(pd.DataFrame.append, res)

# Load all measurement sheets into a single data frame
dt = find_sheets_by_col(None)
dt.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 24592 entries, 0 to 106
Data columns (total 28 columns):
base_deficit_arterial    4094 non-null object
base_excess_arterial     2679 non-null object
blood_gases              6755 non-null object
date                     24588 non-null datetime64[ns]
date/time                24592 non-null datetime64[ns]
hco3                     23 non-null float64
hco3a                    6702 non-null object
icp                      0 non-null float64
icp1                     16183 non-null object
icp2                     8828 non-null object
map                      15862 non-null object
paco2                    6723 non-null object
pao2                     6726 non-null object
pbto2                    24592 non-null float64
pha                      6726 non-null object
pupil_response           1664 non-null object
pupil_response_l         6675 non-null object
pupil_response_r         6643 non-null object
pupil_size               2366 non-null object


#### Account for "unnamed" Columns

In [354]:
dt[~dt['sheet_name'].isin(['1051', '1055', '1046', '1052', '980'])]\
    .set_index('sheet_name')[[c for c in dt if 'unnamed' in c]]\
    .dropna(how='all', axis=0).iloc[:50,:]
# This should be empty if all "Unnamed" column issues have been addressed.
# This involves at least the following updates to the input spreadsheet:
# - Add 'Vital Signs' header for patient 1046
# - Change ICP header for patient 1052 to ICP1 and ICP2

,unnamed:_14,unnamed:_19
sheet_name,,
1051,NaN,11
1051,NaN,10
1046,NaN,9
1046,NaN,15


In [355]:
dt = dt.drop([c for c in dt if 'unnamed' in c], axis=1)

#### Known Issue Adjustments

In [356]:
# Patient 1062 - Pupil Size cell is randomly filled with this string -- so remove it
#dt = dt[dt['pupil_size'] == 'Neurological Assessments']['sheet_name']
dt = dt[dt['pupil_size'] != 'Neurological Assessments']

### Questions

 - What does a pupil size of 'Left - 3mm, Left - 5mm, Right - 2mm, Right - 3mm' mean where there are multiple measurements for each side?
 - Same as above except for Pupil Response values like 'Left - Nonreactive, Left - Fixed, Right - Sluggish'?
 - More serious problems:
     - _Sheet ID_ - _Cell Range_
     - 1028 - Q78
     - 1058 - Q92, R92, S92
     - Pupil size and response extend way beyond dates:
         - 953 - O98:P144
         - 952 - O110:P165
         - 949 - O94:P142
         - 947 - O101:P154
         - 938 - O106:P106
         - 933 - O99:P128
     - Huge sections of ptbo2 and dates completely missing:
         - 951
         - 948
         - 945
         - 944
     - 969 - A2 --> why is UID not equal to number of sheet?
     - 932&967 --> Why are pupil sizes set as dates?
 - Less serious problems:
     - _Sheet ID_ - _Cell Range_
     - 969 - R48 --> why is ICP '-8, 4'?
     - 760 - L5 --> What does it mean to have two values here for "Base Deficit, Arterial" (ie '16, 15')?
     - 1062 - O177 --> Where did "Neurological Assessments" come from?
     - 785 - A4 --> Why is the UID 'Br'?
     - 731 - A1 --> Why is UID column labeled UID100?
     - 542 - B1:B45 --> Why is UID column in here twice?
     - 1063, 1028, 1054 - O1:P1 --> Shouldn't these be "Pupil Size" and "Pupil Response"?
     - 1051 - T72:T73
     - 1055 - V58, V81, V87
     - 980 - T119, T121:T123
     - 812 - G108 --> How did this label get here?
 - Missing Headers:
     - _Sheet ID_ - _Description_
     - 1046 - 'Vital Signs' column header missing
     - 1052 - What is last column supposed to be? ICP2?

In [357]:
dt.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 24592 entries, 0 to 106
Data columns (total 26 columns):
base_deficit_arterial    4094 non-null object
base_excess_arterial     2679 non-null object
blood_gases              6755 non-null object
date                     24588 non-null datetime64[ns]
date/time                24592 non-null datetime64[ns]
hco3                     23 non-null float64
hco3a                    6702 non-null object
icp                      0 non-null float64
icp1                     16183 non-null object
icp2                     8828 non-null object
map                      15862 non-null object
paco2                    6723 non-null object
pao2                     6726 non-null object
pbto2                    24592 non-null float64
pha                      6726 non-null object
pupil_response           1664 non-null object
pupil_response_l         6675 non-null object
pupil_response_r         6643 non-null object
pupil_size               2366 non-null object


## Field Examination

### UID

In [358]:
(dt['uid'] == dt['sheet_name']).value_counts()

True    24592
dtype: int64

In [359]:
sorted(dt[dt['uid'] != dt['sheet_name']]['sheet_name'].unique())

[]

### Pupil Size

In [360]:
def type_counts(x):
    return pd.Series([v.__class__.__name__ for v in x]).value_counts()

In [361]:
dt['pupil_size'].fillna('NA').value_counts()

NA                                                                       22226
Left - 3mm, Right - 3mm                                                    820
Left - 2mm, Right - 2mm                                                    567
Left - 4mm, Right - 4mm                                                    176
Left - 2mm, Right - 4mm                                                     77
Left - 2mm, Right - 3mm                                                     73
Left - 5mm, Right - 2mm                                                     71
Left - 6mm, Right - 4mm                                                     61
Left - 8mm, Right - 6mm                                                     60
Left - 8mm, Right - 5mm                                                     47
Left - 3mm, Right - 5mm                                                     35
Left - 4mm, Right - 3mm                                                     32
Left - 8mm, Right - 3mm                             

In [362]:
dt['pupil_size_l'].fillna('NA').value_counts()

NA                  17897
3.0                  2113
2.0                  1471
3mm                   944
4.0                   568
2mm                   559
4mm                   252
5.0                   240
6.0                   113
7.0                    87
UTA                    80
6mm                    68
8.0                    60
1mm                    50
5mm                    26
9.0                    20
Unable to assess       12
1.0                    11
7mm                     8
8mm                     7
UTA                     1
 8mm                    1
 1mm                    1
c 2mm                   1
c 5mm                   1
c 3mm                   1
dtype: int64

In [363]:
dt['pupil_size_r'].fillna('NA').value_counts()

NA                  17897
3.0                  2046
2.0                  1241
3mm                   796
4.0                   532
2mm                   445
4mm                   343
5.0                   341
6.0                   201
8.0                   152
8mm                   100
9.0                    79
1mm                    74
7.0                    72
UTA                    67
Unable to assess       51
6mm                    49
5mm                    40
7mm                    23
1.0                    21
REyeSwl                 7
2.5                     5
c 3mm                   3
Swollen shut            2
 8mm                    1
c 4mm                   1
c 2mm                   1
c 5mm                   1
 1mm                    1
dtype: int64

In [364]:
type_counts(dt['pupil_size'])

float    22226
str       2366
dtype: int64

In [365]:
dprob = dt[dt['pupil_size'].apply(lambda x: not isinstance(x, str) and not pd.isnull(x))]
print('Ids with issues: ', dprob['uid'].unique())
print('Example invalid values:', dprob['pupil_size'].unique()[:3])

Ids with issues:  []
Example invalid values: []


In [411]:
PS_SIZE_REGEX = re.compile('(\d+)mm')
PS_LEFT_REGEX = re.compile('left - (\d+)mm')
PS_RIGHT_REGEX = re.compile('right - (\d+)mm')

def split_pupil_size(d):
    def find_value(x, regex):
        """ Extract numeric pupil size in mm from value (using given regex), if it exists"""
        try:
            return regex.findall(x.lower())[0]
        except:
            return None
    d['pupil_size_l2'] = d['pupil_size'].apply(find_value, args=(PS_LEFT_REGEX,))
    d['pupil_size_r2'] = d['pupil_size'].apply(find_value, args=(PS_RIGHT_REGEX,))
    return d.drop('pupil_size', axis=1)


def convert_pupil_size(x):
    if pd.isnull(x) or isinstance(x, datetime.datetime):
        return None
    try:
        return float(x)
    except:
        try:
            float(PS_SIZE_REGEX.findall(x.lower())[0])
        except:
            return None
        
def prep_pupil_size(d):
    d = split_pupil_size(d)
    d['pupil_size_l'] = d['pupil_size_l'].apply(convert_pupil_size)
    d['pupil_size_r'] = d['pupil_size_r'].apply(convert_pupil_size)
    d['pupil_size_l2'] = d['pupil_size_l2'].apply(convert_pupil_size)
    d['pupil_size_r2'] = d['pupil_size_r2'].apply(convert_pupil_size)
    d['pupil_size_l'] = np.where(d['pupil_size_l'].isnull(), d['pupil_size_l2'], d['pupil_size_l'])
    d['pupil_size_r'] = np.where(d['pupil_size_r'].isnull(), d['pupil_size_r2'], d['pupil_size_r'])
    d['pupil_size_l'] = d['pupil_size_l'].astype(np.float64)
    d['pupil_size_r'] = d['pupil_size_r'].astype(np.float64)
    d = d.drop(['pupil_size_l2', 'pupil_size_r2'], axis=1)
    return d

x = prep_pupil_size(dt.copy())
x[['pupil_size_l', 'pupil_size_r']].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 24592 entries, 0 to 106
Data columns (total 2 columns):
pupil_size_l    7033 non-null float64
pupil_size_r    7045 non-null float64
dtypes: float64(2)
memory usage: 576.4 KB


### Pupil Response

In [367]:
dt['pupil_response'].fillna('NA').value_counts()

NA                                                                22928
Left - Brisk, Right - Brisk                                         853
Left - Nonreactive, Right - Nonreactive                             188
Left - Sluggish, Right - Brisk                                      159
Left - Sluggish, Right - Sluggish                                   152
Left - Brisk, Right - Sluggish                                      103
Left - Fixed, Right - Sluggish                                       42
Left - Fixed, Right - Fixed                                          40
Left - Fixed, Right - Brisk                                          31
Left - Nonreactive, Right - Brisk                                    17
Left - Brisk, Right - Nonreactive                                    17
Left - Nonreactive, Left - Fixed, Right - Sluggish                   17
Left - Brisk, Right - Fixed                                          14
Left - Nonreactive, Right - Sluggish                            

In [369]:
dt['pupil_response_r'].fillna('NA').value_counts()

NA             17949
Brisk           4513
Fixed           1130
Sluggish         675
Nonreactive      232
UTA               74
Brisk             10
Fixed              3
Birsk              3
c Sluggish         1
Sluggish           1
 Fixed             1
dtype: int64

In [370]:
dt['pupil_response_l'].fillna('NA').value_counts()

NA               17917
Brisk             4972
Fixed              821
Sluggish           582
Nonreactive        154
UTA                 81
Sluggish            55
c Nonreactive        2
c Sluggish           1
 Sluggish            1
Brisk                1
Fixed                1
Birsk                1
c Brisk              1
Brsik                1
 Fixed               1
dtype: int64

In [403]:
PR_TYPES = 'sluggish|fixed|nonreactive|uta|brisk|brsik|birsk'
PR_TYPE_REGEX = re.compile('({})'.format(PR_TYPES))
PR_LEFT_REGEX = re.compile('left - ({})'.format(PR_TYPES))
PR_RIGHT_REGEX = re.compile('right - ({})'.format(PR_TYPES))

def split_pupil_response(d):
    def find_value(x, regex):
        """ Extract numeric pupil response category from value (using given regex), if it exists"""
        try:
            return regex.findall(x.lower())[0]
        except:
            return None
    d['pupil_response_l2'] = d['pupil_response'].apply(find_value, args=(PR_LEFT_REGEX,))
    d['pupil_response_r2'] = d['pupil_response'].apply(find_value, args=(PR_RIGHT_REGEX,))
    return d.drop('pupil_response', axis=1)


def convert_pupil_response(x):
    if pd.isnull(x) or isinstance(x, datetime.datetime):
        return None
    try:
        res = PR_TYPE_REGEX.findall(x.lower())[0]
        return {'bikrs': 'brisk'}.get(''.join(sorted(res)), res)
    except:
        return None
        
def prep_pupil_response(d):
    d = split_pupil_response(d)
    d['pupil_response_l'] = d['pupil_response_l'].apply(convert_pupil_response)
    d['pupil_response_r'] = d['pupil_response_r'].apply(convert_pupil_response)
    d['pupil_response_l2'] = d['pupil_response_l2'].apply(convert_pupil_response)
    d['pupil_response_r2'] = d['pupil_response_r2'].apply(convert_pupil_response)
    d['pupil_response_l'] = np.where(d['pupil_response_l'].isnull(), d['pupil_response_l2'], d['pupil_response_l'])
    d['pupil_response_r'] = np.where(d['pupil_response_r'].isnull(), d['pupil_response_r2'], d['pupil_response_r'])
    d = d.drop(['pupil_response_l2', 'pupil_response_r2'], axis=1)
    return d

x = prep_pupil_response(dt.copy())
x[['pupil_response_l', 'pupil_response_r']].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 24592 entries, 0 to 106
Data columns (total 2 columns):
pupil_response_l    8339 non-null object
pupil_response_r    8301 non-null object
dtypes: object(2)
memory usage: 576.4+ KB


In [382]:
dt['pupil_response_r'].isnull().value_counts()

True     17949
False     6643
dtype: int64

In [397]:
# There are some pupil response values that have only left pupil values, not right
dt['pupil_response'].isnull().value_counts()

True     22928
False     1664
dtype: int64

### Blood Gases

In [432]:
type_counts(dt['blood_gases'])

NaTType     14110
datetime     6754
float        3727
str             1
dtype: int64

In [433]:
def prep_blood_gases(d):
    def convert(x):
        r = pd.to_datetime(x)
        return r if isinstance(r, pd.Timestamp) else None
    d['blood_gases'] = d['blood_gases'].apply(convert)
    return d

x = prep_blood_gases(dt.copy())
type_counts(x['blood_gases'])

NaTType      17838
Timestamp     6754
dtype: int64

### Base Deficit Arterial

In [434]:
type_counts(dt['base_deficit_arterial'])

float    24401
int        170
str         21
dtype: int64

In [438]:
def prep_base_deficit(d):
    def convert(x):
        try:
            return float(x)
        except:
            try:
                return float(x.split(',')[0].replace('n', '').strip())
            except:
                return None
    d['base_deficit_arterial'] = d['base_deficit_arterial'].apply(convert)
    return d

x = prep_base_deficit(dt.copy())
type_counts(x['base_deficit_arterial'])

float64    24592
dtype: int64

### Prep

In [412]:
dex = dt.copy()
dex = prep_pupil_size(dex)
dex = prep_pupil_response(dex)
dex.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 24592 entries, 0 to 106
Data columns (total 24 columns):
base_deficit_arterial    4094 non-null object
base_excess_arterial     2679 non-null object
blood_gases              6755 non-null object
date                     24588 non-null datetime64[ns]
date/time                24592 non-null datetime64[ns]
hco3                     23 non-null float64
hco3a                    6702 non-null object
icp                      0 non-null float64
icp1                     16183 non-null object
icp2                     8828 non-null object
map                      15862 non-null object
paco2                    6723 non-null object
pao2                     6726 non-null object
pbto2                    24592 non-null float64
pha                      6726 non-null object
pupil_response_l         8339 non-null object
pupil_response_r         8301 non-null object
pupil_size_l             7033 non-null float64
pupil_size_r             7045 non-null float6

In [418]:
dex['pupil_size_r'].value_counts()

3.0    3068
2.0    1945
4.0     902
5.0     475
6.0     308
8.0     155
9.0      79
7.0      78
1.0      30
2.5       5
dtype: int64

In [16]:
import functools
measures = functools.reduce(pd.DataFrame.append, [d[k] for k in d_sheet])
measures.head()

,"Base Deficit, Arterial","Base Excess, Arterial",Blood Gases,Date,Date/Time,HCO3,HCO3a,ICP,ICP 1,ICP 1,...,Unnamed: 14,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 4,Vital Signs,pHa,right pupil response,right pupil size
0,NaN,NaN,NaN,2012-11-11,2012-11-11 17:00:00,NaN,NaN,NaN,22,NaN,...,NaN,NaN,NaN,NaN,NaN,NaT,2012-11-11 17:00:00,NaN,NaN,NaN
1,NaN,NaN,NaN,2012-11-11,2012-11-11 18:00:00,NaN,NaN,NaN,10,NaN,...,NaN,NaN,NaN,NaN,NaN,NaT,2012-11-11 18:00:00,NaN,NaN,NaN
2,NaN,1,2012-11-11 20:20:00,2012-11-11,2012-11-11 20:00:00,NaN,24,NaN,1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaT,2012-11-11 20:00:00,7.45,NaN,NaN
3,NaN,NaN,NaN,2012-11-11,2012-11-11 21:00:00,NaN,NaN,NaN,7,NaN,...,NaN,NaN,NaN,NaN,NaN,NaT,2012-11-11 21:00:00,NaN,NaN,NaN
4,NaN,NaN,NaN,2012-11-11,2012-11-11 22:00:00,NaN,NaN,NaN,6,NaN,...,NaN,NaN,NaN,NaN,NaN,NaT,2012-11-11 22:00:00,NaN,NaN,NaN
